In [1]:
# import dependecies 
gkey = "YOUR API KEY"

import requests 
import json

from pprint import pprint
import pandas as pd

In [2]:
# read movie list 
file = "pam/movies_list.csv"
movies_df = pd.read_csv(file, encoding='utf8')
movies_df.head()

,Number,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
0,76,2020-12-31,355,75000000.0,0.0,0.0
1,98,2020-12-31,Moonfall,150000000.0,0.0,0.0
2,10,2020-12-31,Hannibal the Conqueror,50000000.0,0.0,0.0
3,37,2020-09-30,Unhinged,29000000.0,0.0,0.0
4,32,2020-07-17,Tenet,224000000.0,0.0,0.0


In [11]:
# filter to 2018 movies
filtered_2018 = movies_df[(movies_df['Release Date'] > '2018-01-01') & (movies_df['Release Date'] < '2018-12-31')]
filtered_2018.head()

,Number,Release Date,Movie,Production Budget,Domestic Gross,Worldwide Gross
78,88,2018-12-25,On the Basis of Sex,20000000.0,24622687.0,3.880361e+07
79,63,2018-12-25,Holmes & Watson,42000000.0,30568743.0,4.192660e+07
80,15,2018-12-25,Destroyer,9000000.0,1533324.0,3.681096e+06
81,4,2018-12-25,Vice,60000000.0,47836282.0,7.088317e+07
82,38,2018-12-21,Aquaman,160000000.0,335061807.0,1.146895e+09


In [12]:
# create list from movie column
movies = filtered_2018['Movie'].tolist()
len(movies)

143

In [13]:
# replace encoding issue 
encode_list = [movie.replace('â\x80\x99', "'").replace("â\x80\x94", "-").replace("Âº", "º").replace("Ã©", "e") for movie in movies]
encode_list

['On the Basis of Sex',
 'Holmes & Watson',
 'Destroyer',
 'Vice',
 'Aquaman',
 'Welcome to Marwen',
 'Second Act',
 'Bumblebee',
 'Mary Poppins Returns',
 'Spider-Man: Into The Spider-Verse 3D',
 'If Beale Street Could Talk',
 'The Mule',
 'Mortal Engines',
 'That Way Madness Lies',
 'Bird Box',
 'Mary Queen of Scots',
 'Ben is Back',
 'The Possession of Hannah Grace',
 'Werk ohne Autor',
 'The Favourite',
 'Robin Hood',
 'Ralph Breaks The Internet',
 'Creed II',
 'Green Book',
 'Instant Family',
 'Fantastic Beasts: The Crimes of Grindelwald',
 'Widows',
 "Dr. Seuss' The Grinch",
 "The Girl in the Spider's Web",
 'Bohemian Rhapsody',
 'The Nutcracker and the Four Realms',
 "Nobody's Fool",
 'Air Strike',
 'London Fields',
 'Suspiria',
 'Indivisible',
 'Can You Ever Forgive Me?',
 'Halloween',
 'First Man',
 'Beautiful Boy',
 'Goosebumps 2: Haunted Halloween',
 'Bad Times at the El Royale',
 'The Happy Prince',
 'The Hate U Give',
 'Venom',
 'A Star is Born',
 'Hevi reissu',
 'Hell Fes

In [14]:
# add Official Trailer for youtube search 
movies_trailer = [m + " Official Trailer" for m in encode_list]

In [15]:
# need to separate movie list into 20 to keep youtube requests small
split = [movies_trailer[x:x+20] for x in range(0, len(movies_trailer), 20)]
split

[['On the Basis of Sex Official Trailer',
  'Holmes & Watson Official Trailer',
  'Destroyer Official Trailer',
  'Vice Official Trailer',
  'Aquaman Official Trailer',
  'Welcome to Marwen Official Trailer',
  'Second Act Official Trailer',
  'Bumblebee Official Trailer',
  'Mary Poppins Returns Official Trailer',
  'Spider-Man: Into The Spider-Verse 3D Official Trailer',
  'If Beale Street Could Talk Official Trailer',
  'The Mule Official Trailer',
  'Mortal Engines Official Trailer',
  'That Way Madness Lies Official Trailer',
  'Bird Box Official Trailer',
  'Mary Queen of Scots Official Trailer',
  'Ben is Back Official Trailer',
  'The Possession of Hannah Grace Official Trailer',
  'Werk ohne Autor Official Trailer',
  'The Favourite Official Trailer'],
 ['Robin Hood Official Trailer',
  'Ralph Breaks The Internet Official Trailer',
  'Creed II Official Trailer',
  'Green Book Official Trailer',
  'Instant Family Official Trailer',
  'Fantastic Beasts: The Crimes of Grindelwald

In [18]:
# 8 lists from total 143, each 20 
m1 = split[0]
m2 = split[1]
m3 = split[2]
m4 = split[3]
m5 = split[4]
m6 = split[5]
m7 = split[6]
m8 = split[7]

print(m8)

['The Commuter Official Trailer', 'Proud Mary Official Trailer', 'Insidious: The Last Key Official Trailer']


In [ ]:
base_url = "https://www.googleapis.com/youtube/v3/"
  
# new dictionary
d = {}

# try to retrieve videoID for movies in m1 first 
for movie in m1: 
    search_url = base_url + ("search?part=snippet&maxResults=1&q={0}&key={1}").format(movie, gkey)
    
    response = requests.get(search_url).json()
    # check response
    pprint(response)
    
    # add to key-value pair of movie and videoID to dictionary
    d[movie] = response["items"][0]["id"]["videoId"]

# create another (same) for loop for m2, m3..etc 

## only run this part if requests above work!

In [ ]:
# create dataframe from dictionary
df = pd.DataFrame.from_dict(list(d.items()))
df = df.rename(columns={0: "movie_trailer", 1: "video_id"})
df

In [ ]:
df["video_title"] = ""
df["channel_name"] = ""
df["channel_id"] = ""
df["comment_count"] = ""
df["view_count"] = ""
df["like_count"] = ""
df["date_published"] = ""

# for loop to retrieve view count, comment count, like count for each video
for index, row in df.iterrows():
    video = row["video_id"]
    index = index
    
    final_url = base_url + ("videos?part=snippet%2C+statistics&id={0}&key={1}").format(video, gkey)
    final_results = requests.get(final_url).json()
    vid_response = final_results["items"]
    
    for i in vid_response:
        t = i.get("snippet").get("title")
        c = i.get("snippet").get("channelTitle")
        d = i.get("snippet").get("channelId")
        m = i.get("snippet").get("publishedAt")

        comment_count = i.get("statistics").get("commentCount")
        view_count = i.get("statistics").get("viewCount")
        like_count = i.get("statistics").get("likeCount")
      
        df.loc[index, "video_title"] = t
        df.loc[index, "channel_name"] = c
        df.loc[index, "channel_id"] = d
        df.loc[index, "comment_count"] = comment_count
        df.loc[index, "view_count"] = view_count
        df.loc[index, "like_count"] = like_count
        df.loc[index, "date_published"] = m
df